In [1]:
import sympy
import casadi as ca
import numpy as np

In [2]:
class SympyToCasadi:
    def __init__(self):
        self.symbols = {}

    def parse(self, f, depth=0, cse=False, verbose=False):
        prs = lambda f: self.parse(f, depth=depth + 1, cse=False, verbose=verbose)
        f_type = type(f)
        if verbose:
            print("-" * depth, f, "type", f_type)
        if cse:
            cse_defs, cse_exprs = sympy.cse(f)
            assert len(cse_exprs) == 1
            ca_cse_defs = {}
            for symbol, subexpr in reversed(cse_defs):
                ca_cse_defs[prs(symbol)] = prs(subexpr)
            f_ca = prs(cse_exprs[0])
            for k, v in ca_cse_defs.items():
                f_ca = ca.substitute(f_ca, k, v)
            for symbol, subexpr in reversed(cse_defs):
                if str(symbol) in self.symbols:
                    self.symbols.pop(str(symbol))
            return f_ca
        if f_type == sympy.core.add.Add:
            s = 0
            for arg in f.args:
                s += prs(arg)
            return s
        elif f_type == sympy.core.mul.Mul:
            prod = 1
            for arg in f.args:
                prod *= prs(arg)
            return prod
        elif f_type == sympy.core.numbers.Integer:
            return int(f)
        elif f_type == sympy.core.power.Pow:
            base, power = f.args
            base_ca = prs(base)
            if type(power) == sympy.core.numbers.Half:
                return ca.sqrt(base_ca)
            else:
                return base_ca ** prs(power)
        elif f_type == sympy.core.symbol.Symbol:
            if str(f) not in self.symbols:
                self.symbols[str(f)] = ca.SX.sym(str(f))
            return self.symbols[str(f)]
        elif f_type == sympy.matrices.dense.MutableDenseMatrix:
            mat = ca.SX(f.shape[0], f.shape[1])
            for i in range(f.shape[0]):
                for j in range(f.shape[1]):
                    mat[i, j] = prs(f[i, j])
            return mat
        elif f_type == sympy.core.numbers.One:
            return 1
        elif f_type == sympy.core.numbers.Zero:
            return 0
        elif f_type == sympy.core.numbers.NegativeOne:
            return -1
        elif f_type == sympy.core.numbers.Half:
            return 0.5
        elif str(f_type) == "sin":
            return ca.sin(prs(f.args[0]))
        elif str(f_type) == "cos":
            return ca.cos(prs(f.args[0]))
        else:
            print("unhandled type", type(f), f)

In [3]:
x = sympy.symbols("x")
A = sympy.Matrix(
    [
        [sympy.sin(x), 2 * x**2, 4 * x**2 * sympy.sin(x)],
        [1 + x, x * sympy.cos(x), 2 * x**2 + 3 / x],
        [2 - x, x, sympy.sqrt(x)],
    ]
)
A
parser = SympyToCasadi()
f_ca = parser.parse(A)
f_ca

SX(@1=2, 
[[sin(x), (@1*sq(x)), ((4*sq(x))*sin(x))], 
 [(1+x), (x*cos(x)), ((@1*sq(x))+(3/x))], 
 [(@1-x), x, sqrt(x)]])

In [4]:
f_ca = parser.parse(A, cse=True)
f_ca

SX(@1=sin(x), @2=2, @3=sq(x), @4=(@2*@3), 
[[@1, @4, ((4*@1)*@3)], 
 [(1+x), (x*cos(x)), (@4+(3/x))], 
 [(@2-x), x, sqrt(x)]])

In [5]:
parser.symbols

{'x': SX(x)}

In [6]:
f_eval = ca.Function("f", [parser.symbols["x"]], [f_ca])

np.array(f_eval(1)) - sympy.N(A.subs(x, 1))

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])